# What is Sorting?
Sorting is the process of arranging a list of items in a particular order (i.e. ascending, descending, etc.)

In [2]:
from manim import *

In [7]:
# Introducing the problem of sorting
class Introduction(Scene):
    def construct(self):
        title = Text("Sorting Algorithms").scale(1.5)
        self.play(Write(title))
        self.wait(1)
        self.play(FadeOut(title))

        problem = Text("Given a list of numbers, \n sort them in ascending order.").scale(1.5)
        self.play(Write(problem))
        self.wait(1)
        self.play(FadeOut(problem))

        self.wait(1)

%manim -ql -v WARNING Introduction

In [14]:
# Define an array object so that when it is animated it is a long rectangle with small squares inside
class Array(Rectangle):
    def __init__(self, values, **kwargs):
        self.values = values # List of values in the array
        self.n = len(values) # Number of elements in the array
        
        # Call the constructor of the parent class (Rectangle)
        # Ensure the rectangle is large enough to contain all squares
        super().__init__(width=self.n, height=1, **kwargs)

        # Initialize the squares and texts
        self.squares = VGroup()  # Group for squares
        self.texts = VGroup()    # Group for text
        
        # Calculate the side length of each square based on the number of elements
        square_side = self.width / self.n

        # Create squares and texts for each value
        for i, value in enumerate(self.values):
            square = Square(side_length=square_side)
            # Position each square within the rectangle
            square.move_to(self.get_left() + RIGHT * square_side * (0.5 + i))
            self.squares.add(square)

            text = Text(str(value), font_size=36)
            text.move_to(square.get_center())
            self.texts.add(text)

        # Add the squares and texts as submobjects
        self.add(self.squares, self.texts)
        
    def update_array(self, new_values):
        """ Update the array with new values. """
        self.values = new_values
        for text, value in zip(self.texts, self.values):
            text.set_text(str(value))
    

In [20]:
class ArrayScene(Scene):
    def construct(self):
        # Create the array object
        array_values = [5, 3, 1, 2, 4]
        array = Array(array_values)
        
        # Center the array in the scene
        array.move_to(ORIGIN)
        
        # Animate the array
        self.play(Write(array))
        
        self.wait()
    
%manim -ql -v WARNING ArrayScene

In [24]:
class LatexArray(Rectangle):
    def __init__(self, n, **kwargs):
        self.n = n  # Number of elements in the array

        # Call the constructor of the parent class (Rectangle)
        super().__init__(width=self.n, height=1, **kwargs)

        # Initialize the squares and texts
        self.squares = VGroup()  # Group for squares
        self.texts = VGroup()    # Group for text

        # Calculate the side length of each square based on the number of elements
        square_side = self.width / self.n

        # Create squares and LaTeX texts for each value
        for i in range(self.n):
            square = Square(side_length=square_side)
            # Position each square within the rectangle
            square.move_to(self.get_left() + RIGHT * square_side * (0.5 + i))
            self.squares.add(square)

            if i == 0:
                latex_text = f"a_{{{i+1}}}"
            #elif i == self.n - 1:
            #    latex_text = f"a_{{{i+1}}}"
            #else:
            #    latex_text = "\\dots"
            else:
                latex_text = f"a_{{{i+1}}}"
            
            text = MathTex(latex_text, font_size=36)
            text.move_to(square.get_center())
            self.texts.add(text)

        # Add the squares and texts as submobjects
        self.add(self.squares, self.texts)

    def update_array(self, new_values):
        """ Update the array with new values. """
        # Implement the logic to update LaTeX text if needed
        pass


In [25]:
class LatexArrayScene(Scene):
    def construct(self):
        # Create the array object
        array = LatexArray(5)

        # Center the array in the scene
        array.move_to(ORIGIN)

        # Animate the array
        self.play(Write(array))

        self.wait()

%manim -ql -v WARNING LatexArrayScene